# TP03 — Inverse Kinematics (Session 2: Exercises 5-8)

**GEII3 — Industrie 4.0: Robotique | Lab Session 3 - Part 2**

Advanced IK: PUMA 560, trajectory tracking, and practical applications.

---

In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
from math import cos, sin, pi, atan2, sqrt, acos, asin, atan
from scipy.optimize import fsolve, minimize
from scipy.linalg import pinv, norm
from mpl_toolkits.mplot3d import Axes3D

try:
    import roboticstoolbox as rtb
    from spatialmath import SE3, SO3
    RTB_AVAILABLE = True
    print(f"✓ RTB {rtb.__version__}")
except ImportError:
    RTB_AVAILABLE = False

# Import helper functions
from tp_02_fk_helpers import (
    fk_dh, fk_ets, rotx, roty, rotz, transx, transy, transz,
    extract_position, extract_rotation
)

from tp_03_ik_helpers import (
    plot_robot_2d, compare_solutions, normalize_angle,
    create_rtb_robot_from_dh, create_rtb_robot_from_ets
)

np.set_printoptions(precision=4, suppress=True)
print("✓ Setup complete")

---

# Exercise 5: PUMA 560 - Analytical IK with Decoupling

**Estimated time**: 80 minutes

## Problem Description

6-DOF PUMA 560 with spherical wrist.

**DH Parameters** (Modified):

| i | $\theta_i$ | $d_i$ | $a_{i-1}$ | $\alpha_{i-1}$ |
|---|-----------|-------|-----------|----------------|
| 1 | $q_1$     | 0     | 0         | 0              |
| 2 | $q_2$     | 0     | 0         | 90°            |
| 3 | $q_3$     | 0     | 0.4318    | 0              |
| 4 | $q_4$     | 0.15  | 0.0203    | -90°           |
| 5 | $q_5$     | 0.4318| 0         | 90°            |
| 6 | $q_6$     | 0     | 0         | -90°           |

## Task 5.1: Position Subproblem (30 min)

**Key concept**: Wrist center position is determined by first 3 joints.

Given desired end-effector pose $T_d$:

1. **Extract desired wrist center**:
   $$\mathbf{p}_w = \mathbf{p}_d - d_6 \cdot R_d \cdot \begin{bmatrix}0\\0\\1\end{bmatrix}$$
   
   where $d_6$ is distance from wrist to end-effector.

2. **Solve for** $(q_1, q_2, q_3)$ to reach $\mathbf{p}_w$

### Your derivation:

(Sketch and equations for first 3 joints)

---

## Task 5.2: Implementation - Position Subproblem (25 min)

In [ ]:
# PUMA 560 parameters (Modified DH)
a2_puma = 0.4318
a3_puma = 0.0203
d3_puma = 0.15
d4_puma = 0.4318

def ik_puma_position(px, py, pz, a2, a3, d3, d4, elbow='up', wrist='up'):
    """
    Solve position subproblem for PUMA 560 (first 3 joints).
    
    Parameters:
    - px, py, pz: wrist center position
    - a2, a3, d3, d4: robot parameters
    - elbow: 'up' or 'down'
    - wrist: 'up' or 'down'
    
    Returns: [q1, q2, q3] or None
    """
    # Joint 1: rotation about base
    q1 = atan2(py, px)
    
    # Distance in XY plane
    r = sqrt(px**2 + py**2)
    
    # Adjust for link offset a3
    r_adj = r - a3
    z_adj = pz
    
    # Effective distance to wrist center
    D = sqrt(r_adj**2 + (z_adj - d3)**2)
    
    # Check reachability
    if D > (a2 + d4) or D < abs(a2 - d4):
        print(f"  Wrist unreachable! D={D:.3f}")
        return None
    
    # Solve for q3 (elbow angle)
    cos_q3 = (D**2 - a2**2 - d4**2) / (2 * a2 * d4)
    cos_q3 = np.clip(cos_q3, -1, 1)
    
    if elbow == 'up':
        q3 = acos(cos_q3)
    else:
        q3 = -acos(cos_q3)
    
    # Solve for q2 (shoulder angle)
    alpha = atan2(z_adj - d3, r_adj)
    beta = atan2(d4 * sin(q3), a2 + d4 * cos(q3))
    q2 = alpha - beta
    
    return np.array([q1, q2, q3])


# Test
print("\n" + "="*70)
print("  PUMA 560 - POSITION SUBPROBLEM")
print("="*70)

# Test wrist positions
wrist_targets = [
    (0.4, 0.2, 0.3, "Front-right"),
    (0.0, 0.5, 0.4, "Front-center"),
    (-0.3, 0.3, 0.2, "Left side"),
]

for px, py, pz, desc in wrist_targets:
    print(f"\n[Wrist target: ({px}, {py}, {pz}) - {desc}]")
    
    q_pos = ik_puma_position(px, py, pz, a2_puma, a3_puma, d3_puma, d4_puma, 'up')
    
    if q_pos is not None:
        print(f"  Solution: q1={np.rad2deg(q_pos[0]):.1f}°, "
              f"q2={np.rad2deg(q_pos[1]):.1f}°, q3={np.rad2deg(q_pos[2]):.1f}°")

## Task 5.3: Orientation Subproblem (25 min)

**Given**: Desired orientation $R_d$ and first 3 joints $(q_1, q_2, q_3)$

**Find**: Last 3 joints $(q_4, q_5, q_6)$ for spherical wrist

**Approach**:
1. Compute $R_0^3$ from first 3 joints
2. Desired wrist orientation: $R_3^6 = (R_0^3)^T \cdot R_d$
3. Extract Euler angles ZYZ from $R_3^6$

---

In [ ]:
def rotation_matrix_zyz_to_euler(R):
    """
    Extract ZYZ Euler angles from rotation matrix.
    Returns [alpha, beta, gamma] corresponding to Rz(α)Ry(β)Rz(γ)
    """
    # Check for gimbal lock
    if abs(R[2, 2]) < 0.9999:
        beta = acos(R[2, 2])
        alpha = atan2(R[1, 2], R[0, 2])
        gamma = atan2(R[2, 1], -R[2, 0])
    else:
        # Gimbal lock case
        if R[2, 2] > 0:  # β ≈ 0
            beta = 0
            alpha = 0
            gamma = atan2(-R[1, 0], R[0, 0])
        else:  # β ≈ π
            beta = pi
            alpha = 0
            gamma = atan2(R[1, 0], R[0, 0])
    
    return np.array([alpha, beta, gamma])


def ik_puma_orientation(R_desired, q1, q2, q3, dh_params):
    """
    Solve orientation subproblem for PUMA wrist.
    
    Parameters:
    - R_desired: desired end-effector orientation (3x3)
    - q1, q2, q3: first 3 joint angles (from position solution)
    - dh_params: full DH table
    
    Returns: [q4, q5, q6]
    """
    # Compute R_0^3 from first 3 joints
    q_pos = np.array([q1, q2, q3])
    dh_first3 = dh_params[:3]
    
    T_03, _ = fk_dh(dh_first3, q_pos)
    R_03 = extract_rotation(T_03)
    
    # Compute required wrist orientation
    R_36 = R_03.T @ R_desired
    
    # Extract ZYZ Euler angles
    # For PUMA: q4=Z, q5=Y, q6=Z
    euler = rotation_matrix_zyz_to_euler(R_36)
    
    q4 = euler[0]
    q5 = euler[1]
    q6 = euler[2]
    
    return np.array([q4, q5, q6])


print("\n" + "="*70)
print("  PUMA 560 - COMPLETE IK (POSITION + ORIENTATION)")
print("="*70)

# Define PUMA DH (simplified for demo)
dh_puma = [
    (0, 0,      0,        0,     'r'),
    (0, 0,      0,        pi/2,  'r'),
    (0, d3_puma, a2_puma, 0,     'r'),
    (0, d4_puma, a3_puma, -pi/2, 'r'),
    (0, 0,      0,        pi/2,  'r'),
    (0, 0,      0,        -pi/2, 'r'),
]

# Target pose
px_target, py_target, pz_target = 0.4, 0.2, 0.3

# Desired orientation (pointing down)
R_target = np.array([
    [0, 0, -1],
    [0, 1,  0],
    [1, 0,  0]
])

# Solve position
q_pos = ik_puma_position(px_target, py_target, pz_target, 
                         a2_puma, a3_puma, d3_puma, d4_puma, 'up')

if q_pos is not None:
    # Solve orientation
    q_ori = ik_puma_orientation(R_target, q_pos[0], q_pos[1], q_pos[2], dh_puma)
    
    # Complete solution
    q_complete = np.concatenate([q_pos, q_ori])
    
    print(f"\nComplete PUMA IK solution:")
    print(f"  Position joints: {np.rad2deg(q_pos).astype(int)}°")
    print(f"  Orientation joints: {np.rad2deg(q_ori).astype(int)}°")
    print(f"  Full: {np.rad2deg(q_complete).astype(int)}°")
    
    # Verify
    T_result, _ = fk_dh(dh_puma, q_complete)
    pos_result = extract_position(T_result)
    R_result = extract_rotation(T_result)
    
    pos_error = np.linalg.norm(pos_result - [px_target, py_target, pz_target])
    ori_error = np.linalg.norm(R_result - R_target, 'fro')
    
    print(f"\nVerification:")
    print(f"  Position error: {pos_error:.2e} m")
    print(f"  Orientation error: {ori_error:.2e}")

---

# Exercise 6: General Numerical IK

**Estimated time**: 70 minutes

## Task 6.1: Scipy-based IK (30 min)

Use scipy.optimize for general robots.

In [ ]:
from scipy.optimize import minimize, least_squares

def ik_scipy_minimize(dh_params, T_target, q_init, method='SLSQP'):
    """
    IK using scipy.optimize.minimize.
    """
    def cost_function(q):
        T_current, _ = fk_dh(dh_params, q)
        error = pose_error(T_current, T_target)
        return np.sum(error**2)  # Least squares
    
    result = minimize(cost_function, q_init, method=method)
    
    return result.x, result.success, result.fun


def ik_scipy_least_squares(dh_params, T_target, q_init):
    """
    IK using scipy.optimize.least_squares (Levenberg-Marquardt).
    """
    def residuals(q):
        T_current, _ = fk_dh(dh_params, q)
        return pose_error(T_current, T_target)
    
    result = least_squares(residuals, q_init, method='lm')
    
    return result.x, result.success, result.cost


print("\n" + "="*70)
print("  SCIPY-BASED IK")
print("="*70)

# Test with 3R planar
L1, L2, L3 = 0.3, 0.3, 0.2
dh_3r = [(0,0,L1,0,'r'), (0,0,L2,0,'r'), (0,0,L3,0,'r')]

# Target
x_t, y_t = 0.4, 0.3
T_target = np.eye(4)
T_target[0, 3] = x_t
T_target[1, 3] = y_t

q_init = np.array([0.5, 0.3, -0.2])

# Method 1: minimize
q_min, success_min, cost_min = ik_scipy_minimize(dh_3r, T_target, q_init)
print(f"\nScipy minimize (SLSQP):")
print(f"  Solution: {np.rad2deg(q_min).astype(int)}°")
print(f"  Success: {success_min}, Cost: {cost_min:.2e}")

# Method 2: least_squares
q_ls, success_ls, cost_ls = ik_scipy_least_squares(dh_3r, T_target, q_init)
print(f"\nScipy least_squares (LM):")
print(f"  Solution: {np.rad2deg(q_ls).astype(int)}°")
print(f"  Success: {success_ls}, Cost: {cost_ls:.2e}")

## Task 6.2: RTB IK Comparison (20 min)

In [ ]:
if RTB_AVAILABLE:
    from roboticstoolbox import DHRobot, RevoluteDH
    
    # PUMA 560
    puma = rtb.models.DH.Puma560()
    
    print("\n" + "="*70)
    print("  RTB IK METHODS COMPARISON")
    print("="*70)
    
    # Target pose
    T_target = SE3(0.5, 0.2, 0.4) * SE3.Ry(pi/4)
    
    print(f"\nTarget pose:")
    print(T_target)
    
    # Method 1: Levenberg-Marquardt (numerical)
    q_lm = puma.ikine_LM(T_target)
    print(f"\nikine_LM (Levenberg-Marquardt):")
    print(f"  q = {np.rad2deg(q_lm.q).astype(int)}°")
    print(f"  Success: {q_lm.success}")
    
    # Method 2: Minimum distance (Sugihara)
    q_min = puma.ikine_min(T_target)
    print(f"\nikine_min (Minimum distance):")
    print(f"  q = {np.rad2deg(q_min.q).astype(int)}°")
    print(f"  Success: {q_min.success}")
    
    # Method 3: Numerical inverse (GN)
    q_gn = puma.ikine_GN(T_target)
    print(f"\nikine_GN (Gauss-Newton):")
    print(f"  q = {np.rad2deg(q_gn.q).astype(int)}°")
    print(f"  Success: {q_gn.success}")
    
    # Verify all solutions
    print(f"\n--- Verification ---")
    for name, q_sol in [('LM', q_lm.q), ('min', q_min.q), ('GN', q_gn.q)]:
        T_reached = puma.fkine(q_sol)
        error = np.linalg.norm(T_target.t - T_reached.t)
        print(f"{name:5s}: position error = {error:.2e} m")
    
else:
    print("RTB not available")

## Task 6.3: Performance Comparison (20 min)

In [ ]:
import time
import pandas as pd

# Compare different IK methods
methods = {
    'Newton-Raphson': lambda dh, T, q0: ik_newton_raphson(dh, T, q0, max_iter=50),
    'Damped LS': lambda dh, T, q0: ik_damped_ls(dh, T, q0, max_iter=50),
    'Scipy minimize': lambda dh, T, q0: ik_scipy_minimize(dh, T, q0),
    'Scipy least_sq': lambda dh, T, q0: ik_scipy_least_squares(dh, T, q0),
}

results = []

# Test on 3R planar
n_trials = 20

print("\n" + "="*70)
print("  IK METHODS PERFORMANCE COMPARISON")
print("="*70)
print(f"\nRunning {n_trials} trials per method...\n")

for method_name, method_func in methods.items():
    times = []
    successes = 0
    errors = []
    
    for trial in range(n_trials):
        # Random target in workspace
        angle = np.random.uniform(0, 2*pi)
        radius = np.random.uniform(0.3, 0.6)
        x_t = radius * cos(angle)
        y_t = radius * sin(angle)
        
        T_tgt = np.eye(4)
        T_tgt[0, 3] = x_t
        T_tgt[1, 3] = y_t
        
        q0 = np.random.uniform(-pi, pi, 3)
        
        # Time execution
        start = time.time()
        try:
            if 'Scipy' in method_name:
                q_sol, success, cost = method_func(dh_3r, T_tgt, q0)
                converged = success
            else:
                q_sol, converged, iters, hist = method_func(dh_3r, T_tgt, q0)
            elapsed = time.time() - start
        except:
            elapsed = 0
            converged = False
        
        times.append(elapsed)
        if converged:
            successes += 1
            T_result, _ = fk_dh(dh_3r, q_sol)
            err = np.linalg.norm(extract_position(T_result)[:2] - [x_t, y_t])
            errors.append(err)
    
    results.append({
        'Method': method_name,
        'Success Rate': f"{100*successes/n_trials:.0f}%",
        'Avg Time (ms)': f"{1000*np.mean(times):.2f}",
        'Avg Error (mm)': f"{1000*np.mean(errors):.3f}" if errors else "N/A"
    })

df_results = pd.DataFrame(results)
print(df_results.to_string(index=False))

print("\n** Observations:**")
print("  - Analytical methods (when applicable) are fastest")
print("  - Damped LS more robust near singularities")
print("  - Scipy methods versatile but slower")

---

# Exercise 7: Trajectory Tracking with IK

**Estimated time**: 50 minutes

## Task 7.1: Cartesian Path Following (30 min)

Generate joint-space trajectory from Cartesian path.

In [ ]:
def cartesian_path_to_joint_trajectory(dh_params, path_points, ik_func, q_init):
    """
    Convert Cartesian path to joint trajectory using IK.
    
    Parameters:
    - dh_params: robot DH table
    - path_points: list of (x, y) target positions
    - ik_func: IK function to use
    - q_init: initial joint configuration
    
    Returns:
    - joint_trajectory: list of joint configurations
    - success: whether all points were reached
    """
    joint_trajectory = [q_init]
    q_current = q_init.copy()
    
    for i, (x, y) in enumerate(path_points):
        # Target pose
        T_target = np.eye(4)
        T_target[0, 3] = x
        T_target[1, 3] = y
        
        # Solve IK using previous solution as initial guess
        q_new, converged, _, _ = ik_func(dh_params, T_target, q_current)
        
        if not converged:
            print(f"  Warning: IK failed at point {i}: ({x:.3f}, {y:.3f})")
            return joint_trajectory, False
        
        joint_trajectory.append(q_new)
        q_current = q_new
    
    return joint_trajectory, True


print("\n" + "="*70)
print("  TRAJECTORY TRACKING")
print("="*70)

# Define circular path
n_points = 50
radius = 0.2
center_x, center_y = 0.4, 0.3

theta_path = np.linspace(0, 2*pi, n_points)
path_x = center_x + radius * np.cos(theta_path)
path_y = center_y + radius * np.sin(theta_path)
path_points = list(zip(path_x, path_y))

# Initial configuration
q_start = np.array([0.2, 0.5, -0.3])

# Generate trajectory
print(f"\nGenerating trajectory for circular path...")
print(f"  Center: ({center_x}, {center_y}), Radius: {radius}")
print(f"  Points: {n_points}")

joint_traj, success = cartesian_path_to_joint_trajectory(
    dh_3r, path_points, ik_newton_raphson, q_start
)

print(f"\nTrajectory generation: {'Success' if success else 'Failed'}")
print(f"  Generated {len(joint_traj)} configurations")

# Visualize
if success:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Joint space
    joint_traj_array = np.array(joint_traj)
    for i in range(3):
        axes[0].plot(np.rad2deg(joint_traj_array[:, i]), 
                     label=f'q{i+1}', linewidth=2)
    axes[0].set_xlabel('Waypoint', fontsize=12)
    axes[0].set_ylabel('Joint Angle (deg)', fontsize=12)
    axes[0].set_title('Joint Trajectory', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Cartesian space
    axes[1].plot(path_x, path_y, 'b--', linewidth=2, label='Desired path')
    
    # Actual path from FK
    actual_x, actual_y = [], []
    for q in joint_traj:
        T, _ = fk_dh(dh_3r, q)
        actual_x.append(T[0, 3])
        actual_y.append(T[1, 3])
    
    axes[1].plot(actual_x, actual_y, 'r-', linewidth=2, label='Actual path')
    axes[1].plot(actual_x[0], actual_y[0], 'go', markersize=10, label='Start')
    axes[1].plot(actual_x[-1], actual_y[-1], 'rs', markersize=10, label='End')
    axes[1].set_xlabel('X (m)', fontsize=12)
    axes[1].set_ylabel('Y (m)', fontsize=12)
    axes[1].set_title('Cartesian Path', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    axes[1].axis('equal')
    
    plt.tight_layout()
    plt.show()
    
    # Path accuracy
    path_error = np.sqrt((np.array(actual_x) - path_x)**2 + 
                         (np.array(actual_y) - path_y)**2)
    print(f"\nPath tracking accuracy:")
    print(f"  Mean error: {np.mean(path_error)*1000:.3f} mm")
    print(f"  Max error: {np.max(path_error)*1000:.3f} mm")

## Task 7.2: Workspace Boundary Handling (20 min)

In [ ]:
def test_workspace_limits(dh_params, ik_func, L1, L2, L3):
    """
    Test IK at workspace boundaries.
    """
    print("\n" + "="*70)
    print("  WORKSPACE BOUNDARY TESTING")
    print("="*70)
    
    max_reach = L1 + L2 + L3
    min_reach = abs(L1 - L2 - L3)
    
    print(f"\nWorkspace limits:")
    print(f"  Min reach: {min_reach:.3f} m")
    print(f"  Max reach: {max_reach:.3f} m")
    
    test_points = [
        (max_reach * 0.95, 0, "95% max reach"),
        (max_reach * 1.05, 0, "Beyond max reach (should fail)"),
        (min_reach * 1.1, 0, "Near min reach"),
        (0.3, 0.3, "Moderate point"),
    ]
    
    q_init = np.array([0, pi/4, -pi/4])
    
    for x, y, desc in test_points:
        T_target = np.eye(4)
        T_target[0, 3] = x
        T_target[1, 3] = y
        
        print(f"\n[{desc}] Target: ({x:.3f}, {y:.3f})")
        
        q_sol, converged, iters, hist = ik_func(dh_params, T_target, q_init, max_iter=100)
        
        if converged:
            T_reached, _ = fk_dh(dh_params, q_sol)
            pos = extract_position(T_reached)
            error = np.linalg.norm(pos[:2] - [x, y])
            print(f"  ✓ Converged in {iters} iterations")
            print(f"  Solution: {np.rad2deg(q_sol).astype(int)}°")
            print(f"  Error: {error*1000:.3f} mm")
        else:
            print(f"  ✗ Failed to converge")
            print(f"  Final error: {hist[-1]:.3f}")

# Run test
test_workspace_limits(dh_3r, ik_newton_raphson, L1, L2, L3)

---

# Exercise 8: Final Analysis & Conclusions

**Estimated time**: 40 minutes

## Task 8.1: Method Selection Guide (20 min)

In [ ]:
import pandas as pd

# Create decision matrix
decision_guide = {
    'Criterion': [
        'Robot Type',
        'Speed Requirement',
        'Accuracy',
        'Multiple Solutions',
        'Near Singularities',
        'Implementation Complexity',
        'Real-time Capability',
        'Robustness'
    ],
    'Analytical/Geometric': [
        '2R, 3R, 6R with spherical wrist',
        'Very Fast (<1ms)',
        'Exact (machine precision)',
        'Can enumerate all',
        'Poor (specific configs fail)',
        'High (requires derivation)',
        'Excellent',
        'Medium'
    ],
    'Numerical (Jacobian)': [
        'Any robot',
        'Fast (~10ms)',
        'Good (sub-mm typical)',
        'Finds one (nearest to init)',
        'Poor without damping',
        'Low (general algorithm)',
        'Good',
        'Medium'
    ],
    'Numerical (Damped LS)': [
        'Any robot',
        'Medium (~20ms)',
        'Good',
        'Finds one',
        'Better (handles singularities)',
        'Low',
        'Good',
        'High'
    ],
    'Numerical (Optimization)': [
        'Any robot',
        'Slow (~50-200ms)',
        'Very Good',
        'Finds one (global opt possible)',
        'Good with constraints',
        'Very Low (use library)',
        'Poor',
        'High'
    ]
}

df_guide = pd.DataFrame(decision_guide)

print("\n" + "="*80)
print("  IK METHOD SELECTION GUIDE")
print("="*80)
print("\n" + df_guide.to_string(index=False))

print("\n\n" + "="*80)
print("  RECOMMENDATIONS FOR GEII ENGINEERS")
print("="*80)
print("""
1. INDUSTRIAL APPLICATIONS (PUMA, ABB, KUKA):
   → Use analytical IK when available (decoupling for 6-DOF)
   → Fallback to damped least squares for edge cases
   → Pre-compute multiple solutions, select based on criteria

2. RESEARCH / CUSTOM ROBOTS:
   → Start with numerical methods (easier to implement)
   → Use scipy.optimize for offline planning
   → Jacobian methods for real-time control

3. TRAJECTORY GENERATION:
   → Use previous solution as initial guess (warm start)
   → Implement joint limits and collision avoidance as constraints
   → Verify continuity of joint trajectories

4. PRODUCTION SYSTEMS:
   → Analytical IK for speed (typ. <0.1ms)
   → Extensive workspace testing
   → Handle singularities explicitly
   → Include safety margins

5. ROS / MODERN FRAMEWORKS:
   → MoveIt! uses KDL (numerical IK)
   → TracIK (combines analytical + numerical)
   → Understand underlying method for debugging
""")

## Task 8.2: Final Reflection Questions (20 min)

### Q1: Why is IK generally harder than FK?

_Your answer:_


### Q2: When would you prefer numerical IK over analytical, even if analytical exists?

_Your answer:_


### Q3: How do you handle multiple IK solutions in practice?

_Your answer:_


### Q4: What role does the initial guess play in numerical IK?

_Your answer:_


### Q5: Describe a real industrial application where IK is critical.

_Your answer:_


---

---

# Lab Summary & Deliverables

## What You've Accomplished (8 hours)

### Session 1:
✅ Analytical IK for 2R and 3R planar robots  
✅ Numerical methods: Newton-Raphson, Damped LS  
✅ 3R anthropomorphic position IK  

### Session 2:
✅ PUMA 560 IK with decoupling (position + orientation)  
✅ Scipy-based general IK  
✅ RTB comparison and validation  
✅ Trajectory generation from Cartesian paths  
✅ Workspace boundary analysis  

---

## Lab Report Checklist

- [ ] **Analytical derivations** for 2R, 3R, PUMA position/orientation
- [ ] **Implementations** of all IK methods
- [ ] **Validation** against RTB
- [ ] **Performance comparison** (speed, accuracy, robustness)
- [ ] **Trajectory examples** with visualizations
- [ ] **Analysis answers** to all questions
- [ ] **Method selection guide** summary
- [ ] **Conclusions** and lessons learned

---

## Grading Criteria

- **Technical correctness** (40%): Proper derivations, working code
- **Conceptual understanding** (30%): Quality of analysis and answers
- **Implementation quality** (20%): Code clarity, efficiency, validation
- **Presentation** (10%): Plots, organization, documentation

---

## Next Steps

After mastering IK, the natural progression is:

1. **Differential Kinematics** - Velocity, acceleration analysis
2. **Trajectory Planning** - Smooth motion generation, time-optimal paths
3. **Dynamics & Control** - Forces, torques, PID control
4. **Advanced Topics** - Redundancy resolution, obstacle avoidance, optimal IK

---

**Congratulations on completing TP03!** 🎓🤖

You now have comprehensive knowledge of inverse kinematics and can apply various solution methods to industrial robots.

**Key Takeaway**: There is no single "best" IK method—the choice depends on robot geometry, application requirements, and computational constraints.

---